In [1]:
from keras.models import load_model

from RodentMRISkullStripping3D.rbm3.core.dice import dice_coef,dice_coef_loss
from RodentMRISkullStripping3D.rbm3.core.utils import min_max_normalization, resample_img
from RodentMRISkullStripping3D.rbm3.core.paras import PreParas, KerasParas
from RodentMRISkullStripping3D.rbm3.scripts.rbm3 import brain_seg_prediction
from RodentMRISkullStripping3D.rbm3.scripts.rbm3 import out_LabelHot_map_3D

from pathlib import Path
import SimpleITK as sitk
import numpy as np
import os

# Custom code for cleaning up clutter.
from skullstrippingutils import get_3d_paras,load_data_folder,get_dataset_partitions,resample_spacing,skull_strip_file


In [6]:
 # Default Parameters Preparation
model_path = r'C:/Users/yoni.browning/OneDrive - Allen Institute/Documents/GitHub/RodentMRISkullStripping/RodentMRISkullStripping3D/rbm3/scripts/rat_brain-3d_unet.hdf5'
#model_path = r'C:\Users\yoni.browning\OneDrive - Allen Institute\Documents\GitHub\RodentMRISkullStripping\yoniModelRetrain\long_train_unet_3d-ROUND3_10Brain.h5'
pre_paras, keras_paras = get_3d_paras(model_path=model_path)

In [3]:
# Load in the pretrained model
model = load_model(keras_paras.model_path,
                         custom_objects={'dice_coef_loss': dice_coef_loss,
                                         'dice_coef': dice_coef})

In [4]:
# Set parameters 
patch_dims = pre_paras.patch_dims
label_dims = pre_paras.patch_label_dims
strides = pre_paras.patch_strides
n_class = pre_paras.n_class

# Choose which augmentation we want
do_resample_augment = False
do_contrast_augment = False
do_transpose_augment = False


# Preprocess the images for fitting
#data_folder = r'D:\MRI\SkullTrainingData'
data_folder = r'D:\MRI\SkullStripTrainingData_noPituitary'
image_folder_list = os.listdir(data_folder)
    
Images = []
Labels = []
for ii, image_folder in enumerate(image_folder_list):
    print(image_folder)
    Img,Lbl = load_data_folder(os.path.join(data_folder,image_folder),use_outline = False)
    # Default resolution patches and labels
    img = sitk.GetArrayFromImage(Img)
    img = min_max_normalization(img)
    Images.append(img)
   
    lbl = sitk.GetArrayFromImage(Lbl)
    Labels.append(lbl)
    
    # This is where we should do some data augmentation
    # Ultimately, these should be functionalized s.t. we can combine them
    if do_resample_augment:
    # (1) small changes in the brain resolution. 
    # B.c. the model is trained explicitly on 64x64x64 image, this effectivly mimics 
    # a smaller or larger brain
        if np.random.rand()<.1:
            re_Img,re_Lbl = resample_spacing(Img,Lbl,[.09,.09,.09])
            Images.append(re_Img)
            Labels.append(re_Lbl)
        if np.random.rand()<.1:
            re_Img,re_Lbl = resample_spacing(Img,Lbl,[.11,.11,.11])
            Images.append(re_Img)
            Labels.append(re_Lbl)
        
    # (2) changes in contrast
    if do_contrast_augment: 
        from skimage.exposure import adjust_gamma
        # Similarly, do gamma adjustment to fiddle with contrast
        g_img = adjust_gamma(img,.9)
        g_img = min_max_normalization(g_img)
        Images.append(g_img)
        Labels.append(lbl)
        
        gImg = sitk.GetImageFromArry()
        gImg.SetSpacing([.01]*3)
        
        re_Img,re_Lbl = resample_spacing(Img,Lbl,[.09,.09,.09])
        Images.append(re_Img)
        Labels.append(re_Lbl)
        
        re_Img,re_Lbl = resample_spacing(gImg,Lbl,[.11,.11,.11])
        Images.append(re_Img)
        Labels.append(re_Lbl)
        
        # Similarly, do gamma adjustment to fiddle with contrast
        g_img = adjust_gamma(img,1.1)
        g_img = min_max_normalization(g_img)
        Images.append(g_img)
        Labels.append(lbl)
        
        gImg = sitk.GetImageFromArry()
        gImg.SetSpacing([.01]*3)
        
        re_Img,re_Lbl = resample_spacing(Img,Lbl,[.09,.09,.09])
        Images.append(re_Img)
        Labels.append(re_Lbl)
        
        re_Img,re_Lbl = resample_spacing(gImg,Lbl,[.11,.11,.11])
        Images.append(re_Img)
        Labels.append(re_Lbl)

    # (3) rotations/transposes.
    if do_transpose_augment:
        remaining_combos = [[0,2,1],[1,0,2],[1,2,0],[2,0,1],[2,1,0]]
        for ii,TTT in enumerate(remaining_combos):
            if np.random.rand()<.1:
                re_Img = sitk.GetArrayFromImage(Img).transpose(TTT)
                re_Lbl = sitk.GetArrayFromImage(Lbl).transpose(TTT)
                Images.append(re_Img)
                Labels.append(re_Lbl)
        if do_resample_augment:
        # (1) small changes in the brain resolution. 
        # B.c. the model is trained explicitly on 64x64x64 image, this effectivly mimics 
        # a smaller or larger brain
            if np.random.rand()<.1:
                re_Img,re_Lbl = resample_spacing(Img,Lbl,[.09,.09,.09])
                re_Img = re_Img.transpose(TTT)
                re_Lbl = re_Lbl.transpose(TTT)
                Images.append(re_Img)
                Labels.append(re_Lbl)
            if np.random.rand()<.1:
                REImg,RELbl = resample_spacing(Img,Lbl,[.11,.11,.11])
                re_Img = re_Img.transpose(TTT)
                re_Lbl = re_Lbl.transpose(TTT)
                Images.append(re_Img)
                Labels.append(re_Lbl)


660948
D:\MRI\SkullStripTrainingData_noPituitary\660948\660948_manual_skull_strip.seg.nrrd
661275
D:\MRI\SkullStripTrainingData_noPituitary\661275\661275_manual_skull_strip.seg.nrrd
661276
D:\MRI\SkullStripTrainingData_noPituitary\661276\661276_manual_skull_strip.seg.nrrd
661277
D:\MRI\SkullStripTrainingData_noPituitary\661277\661277_manual_skull_strip.seg.nrrd
662198
D:\MRI\SkullStripTrainingData_noPituitary\662198\662199_manual_skull_strip.seg.nrrd
664484
D:\MRI\SkullStripTrainingData_noPituitary\664484\664484_manual_skull_strip.seg.nrrd
674371
D:\MRI\SkullStripTrainingData_noPituitary\674371\675371_manual_skull_strip.seg.nrrd
675329
D:\MRI\SkullStripTrainingData_noPituitary\675329\675329_manual_skull_strip.seg.nrrd
675370
D:\MRI\SkullStripTrainingData_noPituitary\675370\675370_manual_skull_strip.seg.nrrd
675372
D:\MRI\SkullStripTrainingData_noPituitary\675372\685372_manual_skull_strip.seg.nrrd
679922
D:\MRI\SkullStripTrainingData_noPituitary\679922\679922_manual_skull_strip.seg.nrrd

In [5]:
# Load the data
patches = []
labels = []

    
# Prep data for training
for ii,img in enumerate(Images):
    lbl = Labels[ii]
    # Load the Image & Labels, convert to numpy
    length, col, row = img.shape
    categorical_map = np.zeros((n_class, length, col, row), dtype=np.uint8)
    likelihood_map = np.zeros((length, col, row), dtype=np.float32)
    counter_map = np.zeros((length,col,row), dtype=np.float32)
    
    for i in range(0, length-patch_dims[0]+1, strides[0]):
        for j in range(0, col-patch_dims[1]+1, strides[1]):
            for k in range(0, row-patch_dims[2]+1, strides[2]):
                cur_patch=img[i:i+patch_dims[0],
                              j:j+patch_dims[1],
                              k:k+patch_dims[2]][:].reshape([1,1,
                                                             patch_dims[0],
                                                             patch_dims[1],
                                                             patch_dims[2]])
                cur_patch = np.transpose(cur_patch, (0, 2, 3, 4, 1))

                cur_labels=lbl[i:i+patch_dims[0],
                              j:j+patch_dims[1],
                              k:k+patch_dims[2]][:].reshape([1,1,
                                                             patch_dims[0],
                                                             patch_dims[1],
                                                             patch_dims[2]])
                cur_labels = np.transpose(cur_labels, (0, 2, 3, 4, 1))

                patches.append(cur_patch)
                labels.append(cur_labels)

In [6]:
model.optimizer.learning_rate = 1E-5 # Default was 1e-10-->this is a big speed up.

In [7]:
# Split training/validation data. No test data is returned.
train_x,train_y,val_x,val_y,_,_ = get_dataset_partitions(patches,labels) 


In [8]:
# Retrain the model; This gets its own cell so that we can see outputs print :/
history  = model.fit(x = np.vstack(train_x),y = np.vstack(train_y),
                     batch_size=2,
                     epochs=35,
                     verbose=1,
                     validation_data =(np.vstack(val_x),np.vstack(val_y)))


Epoch 1/35
1350/1350 [==============================] - 2481s 2s/step - loss: -0.8963 - dice_coef: 0.8963 - val_loss: -0.9393 - val_dice_coef: 0.9393
Epoch 2/35
1350/1350 [==============================] - 2470s 2s/step - loss: -0.9331 - dice_coef: 0.9331 - val_loss: -0.9562 - val_dice_coef: 0.9563
Epoch 3/35
1350/1350 [==============================] - 2474s 2s/step - loss: -0.9502 - dice_coef: 0.9502 - val_loss: -0.9729 - val_dice_coef: 0.9729
Epoch 4/35
1350/1350 [==============================] - 2463s 2s/step - loss: -0.9602 - dice_coef: 0.9602 - val_loss: -0.9790 - val_dice_coef: 0.9790
Epoch 5/35
1350/1350 [==============================] - 2462s 2s/step - loss: -0.9663 - dice_coef: 0.9663 - val_loss: -0.9820 - val_dice_coef: 0.9820
Epoch 6/35
1350/1350 [==============================] - 2464s 2s/step - loss: -0.9663 - dice_coef: 0.9663 - val_loss: -0.9844 - val_dice_coef: 0.9844
Epoch 7/35
1350/1350 [==============================] - 2465s 2s/step - loss: -0.9745 - dice_coef: 0

In [2]:
# Save the output.
save_path  =r'C:\Users\yoni.browning\OneDrive - Allen Institute\Documents\GitHub\RodentMRISkullStripping\yoniModelRetrain\long_train_unet_3d-V2_nopituitary_15brain.h5'
#model.save(save_path)

In [7]:
this_model_path=model_path 

# Skull strip an individual file.#
mouse = ['684810']

for ii,ms in enumerate(mouse):
    #input_path = r'Y:/ephys/persist/data/MRI/processed/'+ms+'/' + ms + '_100.nii.gz'
    input_path = r'Y:\ephys\persist\data\MRI\processed\100iso_hutch\100iso.nii.gz'
    #output_path =  r'Y:/ephys/persist/data/MRI/processed/' + ms+'/' +ms +'_auto_skull_strip_long_train_unet_3d-V2_nopituitary_15brain.nrrd'
    output_path = r'Y:\ephys\persist\data\MRI\processed\100iso_hutch\100iso_default.nrrd'

    mask,img = skull_strip_file(input_path,output_path,model_path = this_model_path)

Number of iterations:291
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
2

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])


In [9]:
this_model_path

'C:\\Users\\yoni.browning\\OneDrive - Allen Institute\\Documents\\GitHub\\RodentMRISkullStripping\\yoniModelRetrain\\long_train_unet_3d-V2_nopituitary_11brain_v2.h5'

In [13]:
save_path

'C:\\Users\\yoni.browning\\OneDrive - Allen Institute\\Documents\\GitHub\\RodentMRISkullStripping\\yoniModelRetrain\\long_train_unet_3d-V2_nopituitary_11brain.h5'